In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 60.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=20eeb72d4f42b878014ec2d154daf7ec3d34016731b3cf693c3b81225dafe544
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
sc=SparkContext(appName="BREAST_CANCER_ANALYSIS")

In [ ]:
! echo $PYSPARK_SUBMIT_ARGS

In [ ]:
spark=SparkSession.Builder().getOrCreate() #for building dataframes

In [ ]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data

--2021-11-05 06:34:28--  http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124103 (121K) [application/x-httpd-php]
Saving to: ‘wdbc.data’

wdbc.data           100%[===================>] 121.19K   637KB/s    in 0.2s    

2021-11-05 06:34:28 (637 KB/s) - ‘wdbc.data’ saved [124103/124103]



In [ ]:
!head -n 3 wdbc.data

842302,M,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189
842517,M,20.57,17.77,132.9,1326,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.0186,0.0134,0.01389,0.003532,24.99,23.41,158.8,1956,0.1238,0.1866,0.2416,0.186,0.275,0.08902
84300903,M,19.69,21.25,130,1203,0.1096,0.1599,0.1974,0.1279,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.00615,0.04006,0.03832,0.02058,0.0225,0.004571,23.57,25.53,152.5,1709,0.1444,0.4245,0.4504,0.243,0.3613,0.08758


#1) ID Number
#2) Diagnosis (M= malignant, B= benign)
#3) Features

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer

In [ ]:
data=[]

with open("wdbc.data") as infile:
  for line in infile:
    tokens = line.rstrip("\n").split(",")
    y=tokens[1]
    features = Vectors.dense([float(x) for x in tokens[2:]])

    data.append((y,features))

In [ ]:
inputDF=spark.createDataFrame(data, ["label", "features"])

In [ ]:
inputDF.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    M|[17.99,10.38,122....|
|    M|[20.57,17.77,132....|
|    M|[19.69,21.25,130....|
|    M|[11.42,20.38,77.5...|
|    M|[20.29,14.34,135....|
|    M|[12.45,15.7,82.57...|
|    M|[18.25,19.98,119....|
|    M|[13.71,20.83,90.2...|
|    M|[13.0,21.82,87.5,...|
|    M|[12.46,24.04,83.9...|
|    M|[16.02,23.24,102....|
|    M|[15.78,17.89,103....|
|    M|[19.17,24.8,132.4...|
|    M|[15.85,23.95,103....|
|    M|[13.73,22.61,93.6...|
|    M|[14.54,27.54,96.7...|
|    M|[14.68,20.13,94.7...|
|    M|[16.13,20.68,108....|
|    M|[19.81,22.15,130....|
|    B|[13.54,14.36,87.4...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
stringIndexer=StringIndexer(inputCol= "label", outputCol= "labelIndexed")
si_model = stringIndexer.fit(inputDF)
inputDF2 = si_model.transform(inputDF) 

In [ ]:
inputDF2.show()

+-----+--------------------+------------+
|label|            features|labelIndexed|
+-----+--------------------+------------+
|    M|[17.99,10.38,122....|         1.0|
|    M|[20.57,17.77,132....|         1.0|
|    M|[19.69,21.25,130....|         1.0|
|    M|[11.42,20.38,77.5...|         1.0|
|    M|[20.29,14.34,135....|         1.0|
|    M|[12.45,15.7,82.57...|         1.0|
|    M|[18.25,19.98,119....|         1.0|
|    M|[13.71,20.83,90.2...|         1.0|
|    M|[13.0,21.82,87.5,...|         1.0|
|    M|[12.46,24.04,83.9...|         1.0|
|    M|[16.02,23.24,102....|         1.0|
|    M|[15.78,17.89,103....|         1.0|
|    M|[19.17,24.8,132.4...|         1.0|
|    M|[15.85,23.95,103....|         1.0|
|    M|[13.73,22.61,93.6...|         1.0|
|    M|[14.54,27.54,96.7...|         1.0|
|    M|[14.68,20.13,94.7...|         1.0|
|    M|[16.13,20.68,108....|         1.0|
|    M|[19.81,22.15,130....|         1.0|
|    B|[13.54,14.36,87.4...|         0.0|
+-----+--------------------+------

**TRAINING & TEST** **DATA** **SPLIT**

In [ ]:
(trainingData, testData) = inputDF2.randomSplit([0.7,0.3], seed= 23)

***TRAINING DECISION TREE***

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
decisionTree = DecisionTreeClassifier(labelCol = "labelIndexed")

In [ ]:
dtmodel = decisionTree.fit(trainingData)

In [ ]:
dtmodel.numNodes

27

In [ ]:
dtmodel.depth

5

In [ ]:
dtmodel.featureImportances

SparseVector(30, {0: 0.0073, 1: 0.0565, 2: 0.0082, 3: 0.0131, 5: 0.0219, 10: 0.0105, 20: 0.7509, 21: 0.018, 24: 0.0134, 26: 0.0106, 27: 0.0897})

In [ ]:
dtmodel.numFeatures

30

In [ ]:
print(dtmodel.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_12a0fff49a38, depth=5, numNodes=27, numClasses=2, numFeatures=30
  If (feature 20 <= 16.765)
   If (feature 27 <= 0.15635)
    If (feature 3 <= 696.25)
     If (feature 24 <= 0.179)
      Predict: 0.0
     Else (feature 24 > 0.179)
      If (feature 0 <= 9.7485)
       Predict: 0.0
      Else (feature 0 > 9.7485)
       Predict: 1.0
    Else (feature 3 > 696.25)
     If (feature 1 <= 13.295)
      Predict: 0.0
     Else (feature 1 > 13.295)
      Predict: 1.0
   Else (feature 27 > 0.15635)
    If (feature 21 <= 23.215)
     If (feature 2 <= 86.945)
      Predict: 0.0
     Else (feature 2 > 86.945)
      Predict: 1.0
    Else (feature 21 > 23.215)
     Predict: 1.0
  Else (feature 20 > 16.765)
   If (feature 1 <= 14.805)
    If (feature 5 <= 0.13035)
     Predict: 0.0
    Else (feature 5 > 0.13035)
     Predict: 1.0
   Else (feature 1 > 14.805)
    If (feature 10 <= 0.18159999999999998)
     Predict: 0.0
    Else (feature 10 > 

In [ ]:
predictions=dtmodel.transform(testData)

In [ ]:
predictions.select('label', 'labelIndexed', 'probability', 'prediction').show()

+-----+------------+--------------------+----------+
|label|labelIndexed|         probability|prediction|
+-----+------------+--------------------+----------+
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|       0.0|
|    B|         0.0|[0.97609561752988...|     

In [ ]:
evaluator=MulticlassClassificationEvaluator(labelCol= "labelIndexed",predictionCol="prediction",metricName= "accuracy")
accuracy= evaluator.evaluate(predictions)

print("Test Error = %g" % (1.0-accuracy))

Test Error = 0.0519481


**RANDOM FOREST**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier,RandomForestClassificationModel

In [ ]:
rfClassifier = RandomForestClassifier(labelCol = "labelIndexed", numTrees= 100)

In [ ]:
rfModel= rfClassifier.fit(trainingData)

In [ ]:
rfModel.featureImportances

SparseVector(30, {0: 0.0549, 1: 0.0149, 2: 0.0264, 3: 0.0583, 4: 0.0072, 5: 0.0084, 6: 0.0259, 7: 0.0942, 8: 0.0044, 9: 0.0024, 10: 0.0198, 11: 0.0029, 12: 0.011, 13: 0.021, 14: 0.003, 15: 0.0049, 16: 0.0028, 17: 0.0055, 18: 0.0033, 19: 0.0034, 20: 0.1591, 21: 0.0205, 22: 0.1324, 23: 0.1317, 24: 0.0116, 25: 0.0119, 26: 0.045, 27: 0.0992, 28: 0.0103, 29: 0.0038})

In [ ]:
rfModel.toDebugString

'RandomForestClassificationModel: uid=RandomForestClassifier_56a019d76932, numTrees=100, numClasses=2, numFeatures=30\n  Tree 0 (weight 1.0):\n    If (feature 20 <= 16.765)\n     If (feature 24 <= 0.179)\n      If (feature 12 <= 4.816)\n       If (feature 26 <= 0.3863)\n        Predict: 0.0\n       Else (feature 26 > 0.3863)\n        If (feature 12 <= 1.9969999999999999)\n         Predict: 1.0\n        Else (feature 12 > 1.9969999999999999)\n         Predict: 0.0\n      Else (feature 12 > 4.816)\n       Predict: 1.0\n     Else (feature 24 > 0.179)\n      Predict: 1.0\n    Else (feature 20 > 16.765)\n     If (feature 15 <= 0.018135)\n      If (feature 10 <= 0.35509999999999997)\n       If (feature 1 <= 16.34)\n        Predict: 0.0\n       Else (feature 1 > 16.34)\n        Predict: 1.0\n      Else (feature 10 > 0.35509999999999997)\n       If (feature 21 <= 19.064999999999998)\n        Predict: 0.0\n       Else (feature 21 > 19.064999999999998)\n        Predict: 1.0\n     Else (feature 1

In [ ]:
predictions= rfModel.transform(testData)
predictions.select('label', 'labelindexed', 'prediction').show()

+-----+------------+----------+
|label|labelindexed|prediction|
+-----+------------+----------+
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
|    B|         0.0|       0.0|
+-----+------------+----------+
only showing top 20 rows



**ACCURACY BY RANDOM FOREST IS THE HIGHEST COMPARED TO SINGLE DECISION TREE**

In [ ]:
evaluator= MulticlassClassificationEvaluator(labelCol = "labelIndexed", predictionCol= "prediction",metricName= "accuracy")
accuracy= evaluator.evaluate(predictions)

print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.025974
